Here I will be working toward developing a script to get at the percentage of the target covered by FLM membrane.

Problem: In this test case we have to work towards developing a method to skeletonize of fill a object with both an external and internal membrane structure. Given the the RBCs are dye labeled they are not filled and have a small hole that generates a internal and external membrane.

Approach: Today I will be attemptng to skeletonize the RBC membrane as an attempt to get a more accurate identification of percent covered.

ChimeraX script:

open X:/Phagocytosis/sRBC/20190614cs1/track_3/Cell_CamB_ch0_stack0053_488nm_0399080msec_0004086727msecAbs_decon.tif;

open X:/Phagocytosis/sRBC/20190614cs1/track_3/Cell_CamA_ch1_stack0053_560nm_0399080msec_0004086727msecAbs_decon.tif;

material dull;

lighting full depthCue false;

windowsize 800 800;

volume #2 color #ffd059 level 101.7 step 2 region 46,55,0,143,130,228 voxelSize 0.1028;

volume #1 color #2b5884 level 79.4 step 2 voxelSize 0.1028;

surface dust #1,2 metric "volume rank" size 1;

view matrix camera 0.54512,0.46357,-0.69853,-26.17,-0.8311,0.40824,-0.37765,-10.874,0.1101,0.78641,0.60781,36.433;

ui tool show Shell;

In [ ]:
from chimerax.color_key import show_key
from chimerax.core import colors
from chimerax.std_commands.wait import wait
from chimerax.core.commands import (BoolArg, Bounded, CmdDesc, ColormapArg,
                                    ColormapRangeArg, Int2Arg, IntArg,
                                    SurfacesArg, StringArg)
from chimerax.core.commands.cli import EnumOf
from chimerax.map.volumecommand import volume
from chimerax.std_commands.cd import (cd)
from os.path import exists
from numpy import (arccos, array, full, inf, isnan, mean, nan, nanmax, nanmean,
                   nanmin, pi, ravel_multi_index, sign, split, sqrt, subtract,
                   count_nonzero, swapaxes, savetxt, column_stack,nansum, nanstd,
                   unique, column_stack, round_, int64, abs, digitize, linspace,
                   zeros, where, delete)
from scipy.ndimage import (binary_dilation, binary_erosion,
                           generate_binary_structure, iterate_structure, gaussian_filter)
from scipy.spatial import KDTree
import numpy
import scipy
import matplotlib.pyplot as plt
import skimage

In [ ]:
s= session
m= s.models
surface=m[1]
to_cell=m[3]
size=(.1028,.1028,.1028)
palette=None
color_range= None
key=False
volume(session, voxel_size= size)

width = size[1]

targetCoor=numpy.array(to_cell.volume.xyz_bounds()[::])


input_stepsx = int64(round_(abs((targetCoor[1,0]-targetCoor[0,0])/(width))))
input_stepsy = int64(round_(abs((targetCoor[1,1]-targetCoor[0,1])/(width))))
input_stepsz = int64(round_(abs((targetCoor[1,2]-targetCoor[0,2])/(width))))

target_imagex= numpy.array(digitize(to_cell.vertices[:,0],linspace(targetCoor[0,0],targetCoor[1,0],input_stepsx)))
target_imagey= numpy.array(digitize(to_cell.vertices[:,1],linspace(targetCoor[0,1],targetCoor[1,1],input_stepsy)))
target_imagez= numpy.array(digitize(to_cell.vertices[:,2],linspace(targetCoor[0,2],targetCoor[1,2],input_stepsz)))

target_Binary= numpy.zeros((numpy.max(input_stepsx),numpy.max(input_stepsy),numpy.max(input_stepsz)))
target_Binary[target_imagex, target_imagey, target_imagez]=1

TargetGaussian=gaussian_filter(target_Binary,sigma=0.2)
targetGausErode1=binary_erosion(TargetGaussian,iterations=1)

cc=skimage.measure.label(targetGausErode1)


Using scripts by bls to determine of target mask has flm membrane in or 2 pixels next to it.

In [ ]:
"""GetImage"""
mask_vol = surface.volume.full_matrix().copy()
image_3d = to_map.volume.full_matrix().copy()
level = surface.volume.maximum_surface_level

"""Maskimage"""
mask = mask_vol >= level
struct_el = iterate_structure(generate_binary_structure(3, 1), 2)
mask_d = binary_dilation(mask, structure=struct_el)
mask_e = binary_erosion(mask, structure=struct_el, iterations=2)
masked = mask_d ^ mask_e
image_3d *= masked

"""GetCoord"""
image_3d = swapaxes(image_3d, 0, 2)
image_coords = array(image_3d.nonzero())
flattened_image = image_3d.flatten()
pixel_indices = ravel_multi_index(image_coords, image_3d.shape)

"""Querry search"""
def _index(index, tree_max):
    """Tree query pads with tree_max if there are no neighbors."""
    index = index[index < tree_max]
    return index

radius=2
init_verts=surface.vertices
to_map=image_coords.T
knn=200
tree = KDTree(to_map)
dist, index = tree.query(init_verts, k=range(
    1, knn), distance_upper_bound=radius, workers=-1)
dist[dist == inf] = None
distance = nanmean(dist, axis=1)
index = array([_index(ind, tree.n) for ind in index], dtype=object)

def local_intensity(flat_img, pixels, index):
    """Measure local mean intensity normalized to mean of all."""
    face_int = array([nanmean(flat_img[pixels[ind]]) for ind in index])
    return face_int/face_int.mean()




Attempt 2:

In [ ]:
surface=session.models[1]

cell=session.models[3]

flm=surface.volume.full_matrix().copy()

rbc=cell.volume.full_matrix().copy()


flm_lvl=surface.volume.maximum_surface_level

rbc_lvl=cell.volume.maximum_surface_level

"""Masking the RBC"""
flm_mask=flm>=flm_lvl

rbc_mask=rbc>=rbc_lvl